<a href="https://colab.research.google.com/github/panimesh14/Natural-Language-Processing-NLP/blob/main/Majority_Voting_Model_for_Text_Classification_of_Sentiment_of_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Majority Voting Model for Sentiment Classification of Movie Reviews: Random Forest Classifier,Logistic Regression,Naive Bayes,SVM

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk

# Data Loading

In [2]:
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
print(movie_reviews)

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


<CategorizedPlaintextCorpusReader in '.../corpora/movie_reviews' (not loaded yet)>


In [9]:

#Word Tokenization and class assignment
corpora= [(list(movie_reviews.words(fileid)),category) for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)]

# Data Exploration

In [12]:
len(corpora)

2000

In [13]:
corpora[0]

(['plot',
  ':',
  'two',
  'teen',
  'couples',
  'go',
  'to',
  'a',
  'church',
  'party',
  ',',
  'drink',
  'and',
  'then',
  'drive',
  '.',
  'they',
  'get',
  'into',
  'an',
  'accident',
  '.',
  'one',
  'of',
  'the',
  'guys',
  'dies',
  ',',
  'but',
  'his',
  'girlfriend',
  'continues',
  'to',
  'see',
  'him',
  'in',
  'her',
  'life',
  ',',
  'and',
  'has',
  'nightmares',
  '.',
  'what',
  "'",
  's',
  'the',
  'deal',
  '?',
  'watch',
  'the',
  'movie',
  'and',
  '"',
  'sorta',
  '"',
  'find',
  'out',
  '.',
  '.',
  '.',
  'critique',
  ':',
  'a',
  'mind',
  '-',
  'fuck',
  'movie',
  'for',
  'the',
  'teen',
  'generation',
  'that',
  'touches',
  'on',
  'a',
  'very',
  'cool',
  'idea',
  ',',
  'but',
  'presents',
  'it',
  'in',
  'a',
  'very',
  'bad',
  'package',
  '.',
  'which',
  'is',
  'what',
  'makes',
  'this',
  'review',
  'an',
  'even',
  'harder',
  'one',
  'to',
  'write',
  ',',
  'since',
  'i',
  'generally',
  'a

In [14]:
corpora[1200]

(['a',
  'common',
  'complaint',
  'amongst',
  'film',
  'critics',
  'is',
  '"',
  'why',
  'aren',
  "'",
  't',
  'there',
  'more',
  'literate',
  'scripts',
  'available',
  '?',
  '"',
  'quiz',
  'show',
  'gives',
  'signs',
  'of',
  'hope',
  'that',
  'the',
  'art',
  'of',
  'writing',
  'isn',
  "'",
  't',
  'dead',
  'in',
  'hollywood',
  'and',
  'that',
  'we',
  'need',
  'not',
  'only',
  'look',
  'to',
  'independent',
  'films',
  'for',
  'thoughtful',
  'content',
  '.',
  'paul',
  'attanasio',
  "'",
  's',
  'script',
  'takes',
  'what',
  'could',
  'have',
  'been',
  'a',
  'tepid',
  'thriller',
  '(',
  'the',
  'quiz',
  'show',
  'scandals',
  'of',
  'the',
  'late',
  '50s',
  ')',
  'and',
  'delivers',
  'a',
  'telling',
  'parable',
  'about',
  'the',
  'emptiness',
  'of',
  'the',
  'post',
  'war',
  'american',
  'dream',
  'and',
  'the',
  'golden',
  'bubble',
  'that',
  'surrounds',
  'and',
  'protects',
  'tv',
  'networks',
 

#Data Preparation

In [15]:
#Stopwords removal
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=set(stopwords.words())
stop_words
corpora_filt=[];#filtered movie review corpus
for rev,category in corpora:
    rev_filt=[];
    for word in rev:
        if word not in stop_words:
           rev_filt.append(word)
    corpora_filt.append((rev_filt,category))
corpora_filt[1200]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


(['common',
  'complaint',
  'film',
  'critics',
  '"',
  "'",
  'literate',
  'scripts',
  'available',
  '?',
  '"',
  'quiz',
  'show',
  'signs',
  'hope',
  'art',
  'writing',
  "'",
  'dead',
  'hollywood',
  'independent',
  'films',
  'thoughtful',
  'content',
  '.',
  'paul',
  'attanasio',
  "'",
  'script',
  'takes',
  'tepid',
  'thriller',
  '(',
  'quiz',
  'show',
  'scandals',
  'late',
  '50s',
  ')',
  'delivers',
  'telling',
  'parable',
  'emptiness',
  'post',
  'american',
  'dream',
  'golden',
  'bubble',
  'surrounds',
  'protects',
  'tv',
  'networks',
  'sponsors',
  '.',
  'film',
  'riddled',
  'telling',
  'symbols',
  '(',
  '.',
  'g',
  '.',
  "'",
  '58',
  'chrysler',
  ',',
  'radio',
  'announcement',
  'sputnik',
  ')',
  'heavy',
  'handed',
  '.',
  'deft',
  'direction',
  'robert',
  'redford',
  'keen',
  'performances',
  'ralph',
  'fiennes',
  ',',
  'john',
  'turturro',
  'rob',
  'morrow',
  'dovetail',
  'perfectly',
  'carefully'

In [16]:
#Randomizing reviews dataset
import random
random.shuffle(corpora_filt)

In [17]:
#Filtering for most frequent words
num_words=2500;#most frequent 2500 words 
all_words_collection=movie_reviews.words();
complete_wordset=[word.lower() for word in all_words_collection]
complete_vocab=nltk.FreqDist(complete_wordset);#returns dictionary of word:frequency
freq_vocab=list(complete_vocab.keys())[:num_words]
freq_vocab

['plot',
 ':',
 'two',
 'teen',
 'couples',
 'go',
 'to',
 'a',
 'church',
 'party',
 ',',
 'drink',
 'and',
 'then',
 'drive',
 '.',
 'they',
 'get',
 'into',
 'an',
 'accident',
 'one',
 'of',
 'the',
 'guys',
 'dies',
 'but',
 'his',
 'girlfriend',
 'continues',
 'see',
 'him',
 'in',
 'her',
 'life',
 'has',
 'nightmares',
 'what',
 "'",
 's',
 'deal',
 '?',
 'watch',
 'movie',
 '"',
 'sorta',
 'find',
 'out',
 'critique',
 'mind',
 '-',
 'fuck',
 'for',
 'generation',
 'that',
 'touches',
 'on',
 'very',
 'cool',
 'idea',
 'presents',
 'it',
 'bad',
 'package',
 'which',
 'is',
 'makes',
 'this',
 'review',
 'even',
 'harder',
 'write',
 'since',
 'i',
 'generally',
 'applaud',
 'films',
 'attempt',
 'break',
 'mold',
 'mess',
 'with',
 'your',
 'head',
 'such',
 '(',
 'lost',
 'highway',
 '&',
 'memento',
 ')',
 'there',
 'are',
 'good',
 'ways',
 'making',
 'all',
 'types',
 'these',
 'folks',
 'just',
 'didn',
 't',
 'snag',
 'correctly',
 'seem',
 'have',
 'taken',
 'pretty',


In [18]:
#Bag of Words Features
def bow_features(doc):#bag of words/bow for each document vector
    doc_words=set(doc);
    features=[]
    #ind=0;
    for word in freq_vocab:
        features.append(int(word in doc_words))
    return features

In [19]:
#Feature Vector Extraction from BOW
featuresets=[(bow_features(rev),category) for (rev,category) in corpora_filt]
featuresets[1]

([0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,


In [20]:
type(featuresets[1])

tuple

In [21]:
#Test Train Split
test_size=0.3;
num_samples=len(featuresets);
test_ind=int(0.3*num_samples);
print(test_ind)

#Tuple unpacking of features and target class
ind=0;
X=[];y=[];
ind=0;
for t in featuresets:
    feat,target=t
    #print(len(target))
    #print(type(target))
    X.append(np.array(feat));
    y.append(np.array(target));
    
    #X[ind]=feat,y[ind]=target;
    #ind=ind+1;
#X=np.array(X);
#y=np.array(y);
y[y=='pos']=1;
y[y=='neg']=0;
#X=np.array(X.tolist());
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.3)
#y=np.array(y);
#Xtrain=X[:-test_ind];
#Xtest=X[-test_ind:];
#ytrain=y[:-test_ind];
#ytest=y[-test_ind:];
#type(featuresets[0][0])

600


#Model Building

In [22]:
#Importing Models for Majority Vote
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [23]:
logreg=LogisticRegression(penalty="l2")
MultiNB=MultinomialNB()
GaussNB=GaussianNB()
BernNB=BernoulliNB()
svc_clf=LinearSVC()
rfc=RandomForestClassifier(n_estimators=100,max_depth=1000)

In [24]:
logreg.fit(Xtrain,ytrain)
yhat_logreg=logreg.predict(Xtest)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [25]:
MultiNB.fit(Xtrain,ytrain)
yhat_MultiNB=MultiNB.predict(Xtest)

In [26]:
GaussNB.fit(Xtrain,ytrain)
yhat_GaussNB=GaussNB.predict(Xtest)

In [27]:
BernNB.fit(Xtrain,ytrain)
yhat_BernNB=BernNB.predict(Xtest)

In [28]:
svc_clf.fit(Xtrain,ytrain)
yhat_svc=svc_clf.predict(Xtest)

In [29]:
rfc.fit(Xtrain,ytrain)
yhat_rfc=rfc.predict(Xtest)

#Model Evaluation & Testing

In [30]:
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
#Logistic Regression 
print("Logistic Regression")
print(classification_report(ytest,yhat_logreg))

Logistic Regression
              precision    recall  f1-score   support

         neg       0.76      0.80      0.78       283
         pos       0.81      0.78      0.79       317

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600



In [32]:

#Multinomial Naive Bayes
print("Multinomial Naive Bayes")
print(classification_report(ytest,yhat_MultiNB))

Multinomial Naive Bayes
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
         neg       0.78      0.78      0.78       283
         pos       0.82      0.80      0.81       317

    accuracy                           0.79       600
   macro avg       0.53      0.53      0.53       600
weighted avg       0.80      0.79      0.80       600



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:

#Gaussian Naive Bayes
print("Gaussian Naive Bayes")
print(classification_report(ytest,yhat_GaussNB))
#Gausaian Naive Bayes Model displays poor recall on the data and results in only ~50-55% accuracy overall.
#f1 score suggests overall performance

Gaussian Naive Bayes
              precision    recall  f1-score   support

         neg       0.49      0.20      0.28       283
         pos       0.53      0.81      0.64       317

    accuracy                           0.52       600
   macro avg       0.51      0.51      0.46       600
weighted avg       0.51      0.52      0.47       600



In [34]:
#Bernoulli Naive Bayes
print("Bernoulli Naive Bayes")
print(classification_report(ytest,yhat_BernNB))
#Despite simplicity of Bernoulli trials, it shows a good performance overall

Bernoulli Naive Bayes
              precision    recall  f1-score   support

         neg       0.75      0.82      0.78       283
         pos       0.82      0.76      0.79       317

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600



In [35]:

#Support Vector Machine
print("Support Vector Machine")
print(classification_report(ytest,yhat_svc))

Support Vector Machine
              precision    recall  f1-score   support

         neg       0.72      0.74      0.73       283
         pos       0.76      0.75      0.75       317

    accuracy                           0.74       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.74      0.74      0.74       600



In [36]:
#Random Forest Classifier
print("Random Forest Classifier")
print(classification_report(ytest,yhat_rfc))

Random Forest Classifier
              precision    recall  f1-score   support

         neg       0.76      0.81      0.79       283
         pos       0.82      0.77      0.80       317

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600



#Majority Voting Model

In [42]:

#Majority Vote Class
from scipy.stats import mode
class MajorityVote:
    def __init__(self,*classifiers):
        self.classifier=classifiers;

    def predict(self,sample,confidence=False):
        vote=[];
        for clf in self.classifier:
            plabel=clf.predict(sample)
            vote.append(plabel)
        if(confidence):
           maj_vote_count=mode(vote)[1];            
           conf=(maj_vote_count/len(vote))*100;
           return mode(vote)[0], conf        
        else:
            return mode(vote)[0]

In [43]:
vote_combined_clf=MajorityVote(logreg,MultiNB,BernNB,svc_clf,rfc);

In [52]:
yhat_mvote= (vote_combined_clf.predict(Xtest)).T
print(yhat_mvote[5:10])

[['neg']
 ['pos']
 ['neg']
 ['pos']
 ['pos']]


In [40]:
print(classification_report(ytest,yhat_mvote))

              precision    recall  f1-score   support

         neg       0.79      0.83      0.81       283
         pos       0.84      0.80      0.82       317

    accuracy                           0.82       600
   macro avg       0.82      0.82      0.82       600
weighted avg       0.82      0.82      0.82       600



In [41]:
#Model metrics for Sentiment classification of reviews by Majority Vote suggest around 80% chances of correct prediction.
#Combined Majority Vote Model gives relatively improved performance in terms of metrics given the usage of maximum criteria
#Gaussian Naive Bayes omitted because of approximately overall average metrics below 60%
#Inclusion of Deep Learning models may enhance performance further